In [1]:
# | default_exp tools.fs_write

In [1]:
# | export 
from pydantic import BaseModel, Field, field_validator, ValidationInfo
from typing import Optional, List, Any
from enum import Enum
from pathlib import Path
import os

class WriteCommand(str, Enum):
    CREATE = "create"
    EDIT = "edit"

class EditOperationType(str, Enum):
    REPLACE = "replace"
    INSERT = "insert"
    APPEND = "append"
    PREPEND = "prepend"
    DELETE_LINES = "delete_lines"

class FsWriteOperation(BaseModel):
    command: WriteCommand = Field(..., description="Operation mode: 'create' to create/overwrite a file, 'edit' for modifying a file (replace, insert, append, prepend, delete_lines).")
    path: str = Field(..., description="File path, e.g., '/project/src/app.py'. Must be a file, not a directory.")
    file_text: Optional[str] = Field(None, description="Content for create, insert, append, prepend operations.")
    operation_type: Optional[EditOperationType] = Field(None, description="Sub-operation for edit mode: replace, insert, append, prepend, delete_lines.")
    old_str: Optional[str] = Field(None, description="Text to replace in edit/replace (regex or substring).")
    new_str: Optional[str] = Field(None, description="Replacement text in edit/replace.")
    insert_line: Optional[int] = Field(None, description="Line number (1-based) for edit/insert.")
    start_line: Optional[int] = Field(None, description="Start line (1-based) for edit/delete_lines.")
    end_line: Optional[int] = Field(None, description="End line (1-based) for edit/delete_lines.")
    regex_mode: Optional[bool] = Field(True, description="True (default) treats old_str as regex in edit/replace; False for substring matching.")
    respect_gitignore: Optional[bool] = Field(True, description="Check if file is ignored by .gitignore (default True; warns but allows edit if False).")
    blocklist_pattern: str = Field("*__pycache__/*|*.lock|*.o|*.pyc|*.class", description="Patterns to block, e.g., cache/lock files.")
    show_diff: Optional[bool] = Field(True, description="Show unified diff in Git style before writing.")
    trusted: Optional[bool] = Field(False, description="Bypass confirmation prompts if True.")
    summary: Optional[str] = Field(None, description="Brief description of the change.")

    @field_validator("path")
    @classmethod
    def validate_path(cls, value: str) -> str:
        if not value:
            value = os.getcwd()
        path_obj = Path(value)
        if path_obj.is_dir():
            raise ValueError(f"Path {value} is a directory; must be a file")
        if not path_obj.parent.exists():
            raise ValueError(f"Parent directory of {value} does not exist")
        return str(path_obj)

    @field_validator("file_text", "operation_type", "old_str", "new_str", "insert_line", "start_line", "end_line")
    @classmethod
    def validate_fields(cls, value: Any, info: ValidationInfo) -> Any:
        command = info.data.get("command")
        operation_type = info.data.get("operation_type")
        if command == WriteCommand.CREATE and info.field_name == "file_text" and value is None:
            raise ValueError("file_text is required for create command")
        if command == WriteCommand.EDIT:
            if info.field_name == "operation_type" and value is None:
                raise ValueError("operation_type is required for edit command")
            if operation_type == EditOperationType.REPLACE and info.field_name in ["old_str", "new_str"] and value is None:
                raise ValueError(f"{info.field_name} is required for edit/replace")
            if operation_type == EditOperationType.INSERT and info.field_name in ["file_text", "insert_line"] and value is None:
                raise ValueError(f"{info.field_name} is required for edit/insert")
            if operation_type in [EditOperationType.APPEND, EditOperationType.PREPEND] and info.field_name == "file_text" and value is None:
                raise ValueError("file_text is required for edit/append or edit/prepend")
            if operation_type == EditOperationType.DELETE_LINES and info.field_name in ["start_line", "end_line"] and value is None:
                raise ValueError(f"{info.field_name} is required for edit/delete_lines")
            if operation_type == EditOperationType.INSERT and info.field_name == "insert_line" and isinstance(value, int) and value < 1:
                raise ValueError("insert_line must be >= 1")
            if operation_type == EditOperationType.DELETE_LINES and info.field_name in ["start_line", "end_line"]:
                if isinstance(value, int) and value < 1:
                    raise ValueError(f"{info.field_name} must be >= 1")
                if info.field_name == "end_line" and isinstance(value, int) and "start_line" in info.data and isinstance(info.data["start_line"], int) and value < info.data["start_line"]:
                    raise ValueError("end_line must be >= start_line")
        return value

class FsWriteParams(BaseModel):
    operations: List[FsWriteOperation] = Field(..., description="List of write operations to perform")

In [2]:
# | export
import os
import re
import logging
import difflib
from pathlib import Path
from typing import Dict, Any

logger = logging.getLogger(__name__)

class FsWriteTool:
    def __init__(self):
        pass

    @staticmethod
    def get_parameters_schema() -> Dict[str, Any]:
        """Return the OpenAI-compatible schema for fs_write."""
        return {
            "type": "function",
            "function": {
                "name": "fs_write",
                "description": "File operations: create, edit, modify files with diff preview, insert at specific lines. Shows changes in git diff format.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "command": {
                            "type": "string",
                            "enum": ["create", "str_replace", "append", "insert", "prepend", "delete_lines"],
                            "description": "Operation: create (new file), str_replace (replace text), append (add to end), insert (add at line), prepend (add to start), delete_lines (remove lines)"
                        },
                        "path": {"type": "string", "description": "File path"},
                        "file_text": {"type": "string", "description": "Content for create, insert, append, prepend operations"},
                        "old_str": {"type": "string", "description": "Text to replace (regex or substring)"},
                        "new_str": {"type": "string", "description": "Replacement text"},
                        "insert_line": {"type": "integer", "description": "Line number to insert at (1-based)"},
                        "start_line": {"type": "integer", "description": "Start line for delete_lines"},
                        "end_line": {"type": "integer", "description": "End line for delete_lines"},
                        "regex_mode": {"type": "boolean", "description": "True (default) treats old_str as regex in str_replace; False for substring matching"},
                        "show_diff": {"type": "boolean", "description": "Show diff of changes"},
                        "summary": {"type": "string", "description": "Brief description of the change"}
                    },
                    "required": ["command", "path"]
                }
            }
        }

    def _generate_diff(self, original: str, new: str, filepath: str) -> str:
        """Generate unified Git-style diff."""
        original_lines = original.splitlines(keepends=True)
        new_lines = new.splitlines(keepends=True)
        diff_lines = list(difflib.unified_diff(
            original_lines,
            new_lines,
            fromfile=f"a/{filepath}",
            tofile=f"b/{filepath}",
            lineterm=""
        ))
        if not diff_lines:
            return f"No changes in {filepath}"
        
        added = sum(1 for line in diff_lines if line.startswith('+') and not line.startswith('+++'))
        removed = sum(1 for line in diff_lines if line.startswith('-') and not line.startswith('---'))
        diff_output = diff_lines + [f"\nSummary: +{added} -{removed} lines"]
        return '\n'.join(diff_output)

    def _apply_operation(self, file_path: str, params: Dict[str, Any]) -> Dict:
        """Apply a single operation in memory and return final content."""
        # Validate path
        path_obj = Path(file_path)
        if path_obj.is_dir():
            return {"error": f"Path {file_path} is a directory; must be a file"}
        if not path_obj.parent.exists():
            return {"error": f"Parent directory of {file_path} does not exist"}

        # Check for binary file
        try:
            with open(file_path, 'rb') as f:
                chunk = f.read(1024)
                if b'\0' in chunk:
                    return {"error": f"Cannot edit {file_path}: Binary file"}
        except (OSError, UnicodeDecodeError) as e:
            if params["command"] != "create":
                return {"error": f"Failed to read {file_path}: {str(e)}"}

        # Validate parameters
        command = params.get("command")
        valid_commands = ["create", "str_replace", "append", "insert", "prepend", "delete_lines"]
        if command not in valid_commands:
            return {"error": f"Invalid command: {command}; must be one of {valid_commands}"}
        
        if command == "create" and not params.get("file_text"):
            return {"error": "file_text is required for create command"}
        if command == "str_replace" and (not params.get("old_str") or not params.get("new_str")):
            return {"error": "old_str and new_str are required for str_replace"}
        if command == "insert" and (not params.get("file_text") or not params.get("insert_line")):
            return {"error": "file_text and insert_line are required for insert"}
        if command in ["append", "prepend"] and not params.get("file_text"):
            return {"error": "file_text is required for append or prepend"}
        if command == "delete_lines" and (not params.get("start_line") or not params.get("end_line")):
            return {"error": "start_line and end_line are required for delete_lines"}
        if command == "insert" and isinstance(params.get("insert_line"), int) and params["insert_line"] < 1:
            return {"error": "insert_line must be >= 1"}
        if command == "delete_lines":
            if isinstance(params.get("start_line"), int) and params["start_line"] < 1:
                return {"error": "start_line must be >= 1"}
            if isinstance(params.get("end_line"), int) and params["end_line"] < 1:
                return {"error": "end_line must be >= 1"}
            if isinstance(params.get("end_line"), int) and isinstance(params.get("start_line"), int) and params["end_line"] < params["start_line"]:
                return {"error": "end_line must be >= start_line"}

        # Read original content
        original_content = ""
        if command != "create":
            try:
                with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                    original_content = f.read()
            except (OSError, UnicodeDecodeError) as e:
                return {"error": f"Failed to read {file_path}: {str(e)}"}

        current_content = original_content
        line_count = len(current_content.splitlines())
        result = {
            "operation": command,
            "status": None,
            "error": None,
            "summary": params.get("summary")
        }

        if command == "create":
            current_content = params.get("file_text", "")
            result["status"] = "created"
        elif command == "str_replace":
            try:
                logger.debug(f"Applying replace: pattern='{params['old_str']}', file={file_path}")
                new_content = re.sub(params["old_str"], params["new_str"], current_content, flags=re.MULTILINE) if params.get("regex_mode", True) else current_content.replace(params["old_str"], params["new_str"])
                if new_content == current_content:
                    result["status"] = "no changes"
                    result["error"] = f"No matches found for {'regex' if params.get('regex_mode', True) else 'string'} pattern '{params['old_str']}' in {file_path} ({line_count} lines). Content snippet:\n{current_content[:200]}..."
                    logger.debug(f"Replace failed: pattern '{params['old_str']}' not found")
                else:
                    result["status"] = "replace"
                    current_content = new_content
            except re.error as e:
                result["error"] = f"Invalid regex pattern '{params['old_str']}' in {file_path}: {str(e)}"
        elif command == "insert":
            lines = current_content.splitlines(keepends=True)
            insert_idx = max(0, min(params["insert_line"] - 1, len(lines)))
            if params["insert_line"] > line_count + 1:
                result["error"] = f"Insert line {params['insert_line']} exceeds file length ({line_count} lines)"
            else:
                lines.insert(insert_idx, params["file_text"])
                current_content = ''.join(lines)
                result["status"] = "insert"
        elif command == "append":
            current_content = current_content + params["file_text"]
            result["status"] = "append"
        elif command == "prepend":
            current_content = params["file_text"] + current_content
            result["status"] = "prepend"
        elif command == "delete_lines":
            lines = current_content.splitlines(keepends=True)
            start_idx = max(0, params["start_line"] - 1)
            end_idx = min(len(lines), params["end_line"])
            if start_idx >= len(lines):
                result["error"] = f"Delete start line {params['start_line']} exceeds file length ({len(lines)} lines)"
            elif end_idx < start_idx:
                result["error"] = f"Delete end line {params['end_line']} is less than start line {params['start_line']}"
            else:
                current_content = ''.join(lines[:start_idx] + lines[end_idx:])
                result["status"] = "delete_lines"

        return {"content": current_content, "result": result, "original_content": original_content}

    def execute(self, params: Dict[str, Any]) -> Dict:
        """Execute a single file write operation."""
        # Validate required fields
        if not isinstance(params, dict):
            return {
                "success": False,
                "data": [],
                "error": {"type": "InvalidInput", "message": "Params must be a dictionary"}
            }
        if "command" not in params or "path" not in params:
            return {
                "success": False,
                "data": [],
                "error": {"type": "InvalidInput", "message": "command and path are required"}
            }

        file_path = params["path"]
        result = {"command": params["command"], "path": file_path, "data": None, "error": None}

        # Apply operation in memory
        apply_result = self._apply_operation(file_path, params)
        if "error" in apply_result:
            result["error"] = {"type": "EditError", "message": apply_result["error"]}
            return {"success": False, "data": [result], "error": result["error"]}

        new_content = apply_result["content"]
        original_content = apply_result["original_content"]

        # Show diff if requested
        if params.get("show_diff", True):
            diff = self._generate_diff(original_content, new_content, file_path)
            print(diff)

        # Skip confirmation if no changes
        if apply_result["result"]["status"] == "no changes" or apply_result["result"].get("error"):
            result["data"] = {
                "path": file_path,
                "status": apply_result["result"]["status"],
                "size": len(original_content.encode('utf-8')) if original_content else 0,
                "operation": apply_result["result"]
            }
            return {
                "success": True,
                "data": [result],
                "error": None,
                "metadata": {
                    "processed_files": 0,
                    "description": "No changes applied; no files written."
                }
            }

        # Prompt for confirmation
        confirm = input("Confirm edit? (y/n): ").lower().strip()
        if confirm != 'y':
            result["error"] = {"type": "Cancelled", "message": "Edit cancelled by user"}
            return {"success": False, "data": [result], "error": result["error"]}

        # Write final content
        try:
            path_obj = Path(file_path)
            path_obj.parent.mkdir(parents=True, exist_ok=True)
            with open(file_path, 'w', encoding='utf-8') as f:
                f.write(new_content)
            result["data"] = {
                "path": file_path,
                "status": "edited",
                "size": len(new_content.encode('utf-8')),
                "operation": apply_result["result"]
            }
        except (OSError, PermissionError) as e:
            result["error"] = {"type": "WriteError", "message": f"Failed to write {file_path}: {str(e)}"}

        return {
            "success": result["error"] is None,
            "data": [result],
            "error": result["error"],
            "metadata": {
                "processed_files": 1 if result["error"] is None else 0,
                "description": "Results ready for chaining: Use 'data[].path' for execute_bash (e.g., git add)."
            }
        }

In [3]:
# | hide
tool = FsWriteTool()
params = {
    "command": "str_replace",
    "path": "/home/pranav-pc/projects/applied-GenAI-lab/main.py",
    "old_str": r"def\s+log_processing\s*\(\s*\)\s*:",
    "new_str": "def get_updated_prompt():",
    "regex_mode": True,
    "show_diff": True,
    "summary": "Rename log_processing to get_updated_prompt in main.py"
}
import json
result = tool.execute(params)
print(json.dumps(result, indent=2))

No changes in /home/pranav-pc/projects/applied-GenAI-lab/main.py
{
  "success": true,
  "data": [
    {
      "command": "str_replace",
      "path": "/home/pranav-pc/projects/applied-GenAI-lab/main.py",
      "data": {
        "path": "/home/pranav-pc/projects/applied-GenAI-lab/main.py",
        "status": "no changes",
        "size": 80,
        "operation": {
          "operation": "str_replace",
          "status": "no changes",
          "error": "No matches found for regex pattern 'def\\s+log_processing\\s*\\(\\s*\\)\\s*:' in /home/pranav-pc/projects/applied-GenAI-lab/main.py (4 lines). Content snippet:\nfrom agentic.client import main\n\nif __name__ == \"__main__\":\n    main()\n...",
          "summary": "Rename log_processing to get_updated_prompt in main.py"
        }
      },
      "error": null
    }
  ],
  "error": null,
  "metadata": {
    "processed_files": 0,
    "description": "No changes applied; no files written."
  }
}


In [4]:
# | hide
tool = FsWriteTool()

# Operation 1: Delete line 10 (e.g., remove an outdated import)
params1 = {
    "command": "delete_lines",
    "path": "/home/pranav-pc/projects/applied-GenAI-lab/main.py",
    "start_line": 10,
    "end_line": 10,
    "show_diff": True,
    "summary": "Delete outdated import at line 10"
}

# Operation 2: Replace text (e.g., update a variable name)
params2 = {
    "command": "str_replace",
    "path": "/home/pranav-pc/projects/applied-GenAI-lab/main.py",
    "old_str": "old_variable",
    "new_str": "new_variable",
    "regex_mode": False,
    "show_diff": True,
    "summary": "Replace old_variable with new_variable"
}

# Operation 3: Insert text at line 55 (e.g., add a comment)
params3 = {
    "command": "insert",
    "path": "/home/pranav-pc/projects/applied-GenAI-lab/main.py",
    "file_text": "# Added for logging configuration\n",
    "insert_line": 55,
    "show_diff": True,
    "summary": "Insert comment at line 10"
}

# Operation 4: Add function at line 100 (e.g., new function)
params4 = {
    "command": "insert",
    "path": "/home/pranav-pc/projects/applied-GenAI-lab/main.py",
    "file_text": """def log_processing(data):
    logger.debug(f'Processing data: {data}')
    return data\n""",
    "insert_line": 2,
    "show_diff": True,
    "summary": "Insert log_processing function at line 100"
}

import json

# Execute operations sequentially
results = []
for params in [params1, params2, params3, params4]:
    result = tool.execute(params)
    results.append(result)
    print(f"\nResult for {params['summary']}:")
    print(json.dumps(result, indent=2))

# Print combined results
print("\nAll operations results:")
print(json.dumps(results, indent=2))

No changes in /home/pranav-pc/projects/applied-GenAI-lab/main.py

Result for Delete outdated import at line 10:
{
  "success": true,
  "data": [
    {
      "command": "delete_lines",
      "path": "/home/pranav-pc/projects/applied-GenAI-lab/main.py",
      "data": {
        "path": "/home/pranav-pc/projects/applied-GenAI-lab/main.py",
        "status": null,
        "size": 80,
        "operation": {
          "operation": "delete_lines",
          "status": null,
          "error": "Delete start line 10 exceeds file length (4 lines)",
          "summary": "Delete outdated import at line 10"
        }
      },
      "error": null
    }
  ],
  "error": null,
  "metadata": {
    "processed_files": 0,
    "description": "No changes applied; no files written."
  }
}
No changes in /home/pranav-pc/projects/applied-GenAI-lab/main.py

Result for Replace old_variable with new_variable:
{
  "success": true,
  "data": [
    {
      "command": "str_replace",
      "path": "/home/pranav-pc/project

Confirm edit? (y/n):  n



Result for Insert log_processing function at line 100:
{
  "success": false,
  "data": [
    {
      "command": "insert",
      "path": "/home/pranav-pc/projects/applied-GenAI-lab/main.py",
      "data": null,
      "error": {
        "type": "Cancelled",
        "message": "Edit cancelled by user"
      }
    }
  ],
  "error": {
    "type": "Cancelled",
    "message": "Edit cancelled by user"
  }
}

All operations results:
[
  {
    "success": true,
    "data": [
      {
        "command": "delete_lines",
        "path": "/home/pranav-pc/projects/applied-GenAI-lab/main.py",
        "data": {
          "path": "/home/pranav-pc/projects/applied-GenAI-lab/main.py",
          "status": null,
          "size": 80,
          "operation": {
            "operation": "delete_lines",
            "status": null,
            "error": "Delete start line 10 exceeds file length (4 lines)",
            "summary": "Delete outdated import at line 10"
          }
        },
        "error": null
      